# Action Time Measurements

This notebook contains all measurements needed in order to determine the action within the simulation which affect it's performance.

In [1]:
import statsmodels.stats.power as smp

# Set parameters for the power analysis
alpha = 0.05  # Desired level of significance
effect_size = 0.5  # Desired effect size
power = 0.8  # Desired statistical power

# Perform power analysis
n = smp.NormalIndPower().solve_power(effect_size=effect_size, alpha=alpha, power=power)

# Round up the sample size to the nearest whole number
sample_size = int(round(n))

# Output the estimated sample size
print("Estimated sample size:", sample_size)


Estimated sample size: 63


After gathering the samples, let's process them:

In [2]:
import numpy as np
import pandas as pd
# import the '../data/raw/fetch_times_samples.csv' file, import location column as string:
samples_df = pd.read_csv('../data/raw/fetch_times_samples.csv', dtype={'location': str})
samples_df

,location,fetch_tool,horizontal_movement_time,vertical_movement_time,remove_from_shelf_time
0,010303,PALLET_JACK,15.605808,NaN,37.12
1,010305,PALLET_JACK,16.381467,NaN,22.00
2,010306,PALLET_JACK,12.381467,NaN,19.16
3,010403,PALLET_JACK,19.555808,NaN,18.36
4,010501,PALLET_JACK,13.671962,NaN,33.56
...,...,...,...,...,...
187,602203,REACH_FORK,21.262893,2.586090,21.72
188,602303,REACH_FORK,23.258607,2.149789,31.48
189,602403,REACH_FORK,23.768607,2.637050,28.54
190,602503,REACH_FORK,25.074321,2.039587,36.08


In [3]:
# import from the '../src/data/cells.csv' file:
cells = pd.read_csv('../src/data/cells.csv', usecols=['location', 'aisle', 'x_length', 'z_height', 'distance_from_io_to_aisle', 'fetch_tool'])
cells

,location,aisle,x_length,z_height,distance_from_io_to_aisle,fetch_tool
0,010101,1,1.275000,0.0,34.79875,PALLET_JACK
1,010102,1,1.366071,0.0,34.79875,PALLET_JACK
2,010103,1,1.275000,0.0,34.79875,PALLET_JACK
3,010104,1,1.275000,0.0,34.79875,PALLET_JACK
4,010105,1,1.366071,0.0,34.79875,PALLET_JACK
...,...,...,...,...,...,...
5664,FLOOR01,100,4.781250,0.0,22.78625,PALLET_JACK
5665,FLOOR02,100,14.343750,0.0,22.78625,PALLET_JACK
5666,FLOOR03,100,23.906250,0.0,22.78625,PALLET_JACK
5667,FLOOR04,100,33.468750,0.0,22.78625,PALLET_JACK


In [4]:
# merge the two dataframes:
samples_df = pd.merge(samples_df, cells, on=['location', 'fetch_tool'])
samples_df

,location,fetch_tool,horizontal_movement_time,vertical_movement_time,remove_from_shelf_time,aisle,x_length,z_height,distance_from_io_to_aisle
0,010303,PALLET_JACK,15.605808,NaN,37.12,1,3.825000,0.00,34.79875
1,010305,PALLET_JACK,16.381467,NaN,22.00,1,4.098214,0.00,34.79875
2,010306,PALLET_JACK,12.381467,NaN,19.16,1,4.098214,0.00,34.79875
3,010403,PALLET_JACK,19.555808,NaN,18.36,1,3.825000,0.00,34.79875
4,010501,PALLET_JACK,13.671962,NaN,33.56,1,6.375000,0.00,34.79875
...,...,...,...,...,...,...,...,...,...
187,602203,REACH_FORK,21.262893,2.586090,21.72,60,26.775000,2.10,36.23625
188,602303,REACH_FORK,23.258607,2.149789,31.48,60,29.325000,1.75,36.23625
189,602403,REACH_FORK,23.768607,2.637050,28.54,60,29.325000,2.10,36.23625
190,602503,REACH_FORK,25.074321,2.039587,36.08,60,31.875000,1.75,36.23625


In [5]:
samples_df['total_horizontal_distance'] = samples_df['x_length'] + samples_df['distance_from_io_to_aisle']
# calculate the 'horizontal_speed' by dividing the 'total_horizontal_distance' by the 'horizontal_movement_time' columns:
samples_df['horizontal_speed'] = samples_df['total_horizontal_distance'] / samples_df['horizontal_movement_time']
# calculate the 'vertical_speed' by dividing the 'z_height' by the 'vertical_movement_time' columns:
samples_df['vertical_speed'] = samples_df['z_height'] / samples_df['vertical_movement_time']
samples_df

,location,fetch_tool,horizontal_movement_time,vertical_movement_time,remove_from_shelf_time,aisle,x_length,z_height,distance_from_io_to_aisle,total_horizontal_distance,horizontal_speed,vertical_speed
0,010303,PALLET_JACK,15.605808,NaN,37.12,1,3.825000,0.00,34.79875,38.623750,2.474960,NaN
1,010305,PALLET_JACK,16.381467,NaN,22.00,1,4.098214,0.00,34.79875,38.896964,2.374449,NaN
2,010306,PALLET_JACK,12.381467,NaN,19.16,1,4.098214,0.00,34.79875,38.896964,3.141547,NaN
3,010403,PALLET_JACK,19.555808,NaN,18.36,1,3.825000,0.00,34.79875,38.623750,1.975053,NaN
4,010501,PALLET_JACK,13.671962,NaN,33.56,1,6.375000,0.00,34.79875,41.173750,3.011547,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
187,602203,REACH_FORK,21.262893,2.586090,21.72,60,26.775000,2.10,36.23625,63.011250,2.963437,0.812037
188,602303,REACH_FORK,23.258607,2.149789,31.48,60,29.325000,1.75,36.23625,65.561250,2.818795,0.814033
189,602403,REACH_FORK,23.768607,2.637050,28.54,60,29.325000,2.10,36.23625,65.561250,2.758313,0.796344
190,602503,REACH_FORK,25.074321,2.039587,36.08,60,31.875000,1.75,36.23625,68.111250,2.716375,0.858017


In [6]:
import scipy.stats as stats

def test_normality(data):
    """
    Perform Shapiro-Wilk test to determine normality of data.
    
    Arguments:
    data -- Pandas Series containing the data
    
    Returns:
    result -- Boolean indicating whether the data follows a normal distribution
    p_value -- P-value from the Shapiro-Wilk test
    """
    # Perform Shapiro-Wilk test
    _, p_value = stats.shapiro(data)
    
    # Set significance level
    alpha = 0.05
    
    # Determine normality based on p-value and significance level
    if p_value < alpha:
        result = False  # Reject null hypothesis (data does not follow a normal distribution)
    else:
        result = True  # Fail to reject null hypothesis (data may follow a normal distribution)
    
    return result, p_value

In [7]:
def test_normality_for_tools_traits(df: pd.DataFrame, tools: list, traits: list):
  for fetch_tool in tools:
    for trait in traits:
      print(fetch_tool, trait)
      data = df[df['fetch_tool'] == fetch_tool][trait]
      # Test normality
      normality_result, p_value = test_normality(data)

      # Print the result
      if normality_result:
          print("The data may follow a normal distribution.")
      else:
          print("The data does not follow a normal distribution.")

      # Print the p-value
      print("P-value:", p_value)
      print()

In [8]:
all_fetch_tools = samples_df['fetch_tool'].unique()
all_traits = ['remove_from_shelf_time', 'horizontal_speed', 'vertical_speed']
fetch_tools_without_pallet_jack = samples_df['fetch_tool'].unique()[1:]
traits_without_vertical_speed = ['remove_from_shelf_time', 'horizontal_speed']

In [9]:
test_normality_for_tools_traits(samples_df, ['PALLET_JACK'], traits_without_vertical_speed)

PALLET_JACK remove_from_shelf_time
The data may follow a normal distribution.
P-value: 0.9470158219337463

PALLET_JACK horizontal_speed
The data may follow a normal distribution.
P-value: 0.47773072123527527



In [10]:
test_normality_for_tools_traits(samples_df, fetch_tools_without_pallet_jack, all_traits)

ORDER_PICKER remove_from_shelf_time
The data may follow a normal distribution.
P-value: 0.32968056201934814

ORDER_PICKER horizontal_speed
The data may follow a normal distribution.
P-value: 0.0919029638171196

ORDER_PICKER vertical_speed
The data may follow a normal distribution.
P-value: 0.6844973564147949

REACH_FORK remove_from_shelf_time
The data may follow a normal distribution.
P-value: 0.3203481137752533

REACH_FORK horizontal_speed
The data may follow a normal distribution.
P-value: 0.7431575655937195

REACH_FORK vertical_speed
The data may follow a normal distribution.
P-value: 0.19742633402347565



In [11]:
# group by the 'fetch_tool' column and get the mean and std values for the 'horizontal_speed' and 'vertical_speed' columns:
speeds_mean_and_std = samples_df.groupby('fetch_tool').agg({'horizontal_speed': ['mean', 'std'], 'vertical_speed': ['mean', 'std'], 'remove_from_shelf_time': ['mean', 'std']})
speeds_mean_and_std

horizontal_speed           vertical_speed             
                         mean       std           mean       std   
fetch_tool                                                         
ORDER_PICKER         2.034363  0.301744       0.378752  0.045476  \
PALLET_JACK          2.419037  0.418069            NaN       NaN   
REACH_FORK           2.637309  0.336348       0.806503  0.044432   

             remove_from_shelf_time            
                               mean       std  
fetch_tool                                     
ORDER_PICKER              26.034063  6.916285  
PALLET_JACK               25.478281  6.698394  
REACH_FORK                25.703594  7.192422

In [12]:
# take the speeds_mean_and_std table and make into a single level index table:
speeds_mean_and_std.columns = speeds_mean_and_std.columns.map('_'.join)
# reset the index of the speeds_mean_and_std table:
speeds_mean_and_std.reset_index(inplace=True)
speeds_mean_and_std

,fetch_tool,horizontal_speed_mean,horizontal_speed_std,vertical_speed_mean,vertical_speed_std,remove_from_shelf_time_mean,remove_from_shelf_time_std
0,ORDER_PICKER,2.034363,0.301744,0.378752,0.045476,26.034063,6.916285
1,PALLET_JACK,2.419037,0.418069,NaN,NaN,25.478281,6.698394
2,REACH_FORK,2.637309,0.336348,0.806503,0.044432,25.703594,7.192422


In [13]:
# export the speeds_mean_and_std table to a csv file:
speeds_mean_and_std.to_csv('../src/data/fetch_tools_speeds_mean_and_std.csv', index=False)